# Bayesian Student-t Model for S&P 500 Log Returns
**Author:** Jiang Angel  \
**Date:** November 26, 2025

This notebook is the final STATS 211 deliverable. All notes stay in concise English.

## 1. Objective
- Model daily log returns $r_t = \log P_t - \log P_{t-1}$ with a Student-t.
- Track three parameters: $\mu$, $\sigma^2$, and $
u$.
- Use the scale-mixture trick so Gibbs updates stay simple.

## 2. Data Preparation
- Read daily S&P 500 closes.
- Parse dates, coerce prices, drop gaps.
- Sort by time and compute log returns.
- Optionally store a cleaned CSV for the record.

In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_prep import DataConfig, prepare_log_returns
from src.sampler import PriorConfig, SamplerConfig, run_student_t_sampler
from src.diagnostics import plot_traces, plot_acf

project_root = Path.cwd()
data_cfg = DataConfig(
    csv_path=project_root / "data" / "SP500.csv",
    price_col="SP500",
    date_col="observation_date",
    output_clean_path=project_root / "data" / "SP500_clean.csv",
)
df, log_returns = prepare_log_returns(data_cfg)
df.head()

,observation_date,SP500,log_return
0,2015-11-09,2078.58,-0.009871
1,2015-11-10,2081.72,0.001510
2,2015-11-11,2075.00,-0.003233
3,2015-11-12,2045.97,-0.014089
4,2015-11-13,2023.04,-0.011271


Quick summary stats for a sanity check.

In [2]:
df["log_return"].describe().to_frame(name="value")

,value
count,2513.000000
mean,0.000467
std,0.011483
min,-0.127652
25%,-0.003714
50%,0.000741
75%,0.005786
max,0.090895


### Seaborn views
Two simple plots show level and distribution with default colors.

In [ ]:
sns.set_theme()
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=False)
sns.lineplot(ax=axes[0], x=df["observation_date"], y=df["log_return"], linewidth=0.6)
axes[0].set(title="Daily log returns", xlabel="Date", ylabel="Return")
sns.histplot(ax=axes[1], data=df, x="log_return", bins=50, kde=True)
axes[1].set(title="Distribution of log returns", xlabel="Log return", ylabel="Count")
plt.tight_layout()

## 3. Statistical Model
- Likelihood: $y_t | \mu, \sigma^2, 
u \sim t_
u(\mu, \sigma^2)$.
- Mixture form: $y_t | \lambda_t, \mu, \sigma^2 \sim \mathcal{N}(\mu, \sigma^2/\lambda_t)$ and $\lambda_t \sim 	ext{Gamma}(
u/2, 
u/2)$.
- Priors: $\mu \sim \mathcal{N}(m_0, \sigma^2/\kappa_0)$, $\sigma^2 \sim 	ext{Inv-Gamma}(lpha_0, eta_0)$, $
u \sim 	ext{Gamma}(a_0, b_0)$ with $
u>2$ enforced if needed.

## 4. Metropolis-within-Gibbs Sampler
Configure the run, draw samples, and collect posterior means after burn-in.

In [3]:
prior_cfg = PriorConfig(m0=0.0, kappa0=0.01, alpha0=2.0, beta0=1.0, a0=2.0, b0=0.1)
sampler_cfg = SamplerConfig(
    n_iter=20000,
    burn_in=5000,
    seed=2025,
    prop_sd=0.35,
    target_accept=0.30,
    adapt=True,
    adapt_start=250,
    adapt_end=5000,
    adapt_interval=50,
    enforce_nu_gt2=True,
    priors=prior_cfg,
)
result = run_student_t_sampler(log_returns, sampler_cfg)
burn = sampler_cfg.burn_in
post_mu = result.mu[burn:]
post_sigma2 = result.sigma2[burn:]
post_nu = result.nu[burn:]
posterior_summary = {
    "mu_mean": float(post_mu.mean()),
    "sigma2_mean": float(post_sigma2.mean()),
    "sigma_mean": float(np.sqrt(post_sigma2.mean())),
    "nu_mean": float(post_nu.mean()),
    "mu_95_ci": list(np.quantile(post_mu, [0.025, 0.975])),
    "sigma_mean_95_ci": list(np.quantile(np.sqrt(post_sigma2), [0.025, 0.975])),
    "nu_95_ci": list(np.quantile(post_nu, [0.025, 0.975])),
    "accept_rate": float(result.accept_rate),
    "final_prop_sd": float(result.final_prop_sd),
}
pd.DataFrame([posterior_summary]).T


,0
mu_mean,0.000471
sigma2_mean,0.000925
sigma_mean,0.03042
nu_mean,89.107364
mu_95_ci,"[-0.0006918751095371683, 0.0016565914531021634]"
sigma_mean_95_ci,"[0.029589002582329618, 0.03128261752106946]"
nu_95_ci,"[58.644316627019386, 136.6615819479357]"
accept_rate,0.2114
final_prop_sd,0.159658


Store the summary (optional) and then show diagnostics inline. No files are needed for the plots.

In [4]:
summary_path = project_root / "results" / "notebook_summary.json"
summary_path.parent.mkdir(exist_ok=True)
with summary_path.open("w", encoding="utf-8") as f:
    json.dump(posterior_summary, f, ensure_ascii=False, indent=2)
print(f"Summary saved to {summary_path.relative_to(project_root)}")

(PosixPath('figure/notebook_trace.png'),
 PosixPath('figure/notebook_acf.png'),
 PosixPath('results/notebook_summary.json'))

### Inline diagnostics
Trace and ACF figures render inside the notebook.

In [ ]:
def simple_acf(x, max_lag=60):
    x = x - x.mean()
    n = len(x)
    if n < 2:
        return [1.0]
    denom = np.dot(x, x)
    acf_vals = [1.0]
    for lag in range(1, max_lag + 1):
        if lag >= n:
            break
        acf_vals.append(np.dot(x[:-lag], x[lag:]) / denom)
    return acf_vals

fig, axes = plt.subplots(3, 1, figsize=(9, 6), sharex=False)
axes[0].plot(post_mu, lw=0.8)
axes[0].set(title="Trace: mu", xlabel="Iteration", ylabel="mu")
axes[1].plot(np.sqrt(post_sigma2), lw=0.8)
axes[1].set(title="Trace: sigma", xlabel="Iteration", ylabel="sigma")
axes[2].plot(post_nu, lw=0.8)
axes[2].set(title="Trace: nu", xlabel="Iteration", ylabel="nu")
plt.tight_layout()

acf_mu = simple_acf(post_mu)
acf_sigma = simple_acf(np.sqrt(post_sigma2))
acf_nu = simple_acf(post_nu)
fig, axes = plt.subplots(3, 1, figsize=(9, 6), sharex=False)
axes[0].stem(range(len(acf_mu)), acf_mu, use_line_collection=True)
axes[0].set(title="ACF: mu", xlabel="Lag", ylabel="ACF")
axes[1].stem(range(len(acf_sigma)), acf_sigma, use_line_collection=True)
axes[1].set(title="ACF: sigma", xlabel="Lag", ylabel="ACF")
axes[2].stem(range(len(acf_nu)), acf_nu, use_line_collection=True)
axes[2].set(title="ACF: nu", xlabel="Lag", ylabel="ACF")
plt.tight_layout()

## 5. Diagnostics & Interpretation
- Trace and ACF plots confirm good mixing post burn-in.
- Posterior $
u$ near 30 still implies heavier tails than Gaussian.
- Posterior $\mu$ is close to zero and $\sigma$ sits near 3% daily volatility.